In [1]:
import pandas as pd
import json

In [2]:
dir = 'source/raw_trump_tweets_2.json'

df = pd.read_json(open(dir, 'r'))

In [4]:
print(df['text'])
    

0       Jobless Claims fell to their lowest level in 4...
1       Consumer confidence hits an 18 year high, clos...
2       The Democrats are playing a high level CON GAM...
3       “These law enforcement people took the law int...
4       73rd Session of the United Nations General Ass...
5       “Consumer confidence rose in September, notchi...
6       “Remarks by President Trump at a Luncheon Host...
7       “Remarks by President Trump to the 73rd Sessio...
8       Rush Limbaugh to Republicans: “You can kiss th...
9         Thank you Dr. Jeffress! https://t.co/cm7NjQh0ti
10                Thank you Mark! https://t.co/NVGs6WbQe2
11                                https://t.co/h43dehf0WV
12      Will be speaking at the United Nations this mo...
13      Despite requests, I have no plans to meet Iran...
14      Republican Party Favorability is the highest i...
15      The Democrats are working hard to destroy a wo...
16                                 REMEMBER THE MIDTERMS!
17            

In [5]:
df['pre_clean_len'] = [len(t) for t in df.text]

In [19]:
df['pre_clean_len']

0        54
1       140
2       139
3       114
4        81
5       140
6       121
7       137
8       119
9        47
10       39
11       23
12      139
13      139
14      109
15      140
16       22
17       23
18      140
19      140
20      112
21      139
22       71
23      140
24       86
25      121
26      140
27      140
28      139
29       85
       ... 
3169    140
3170    140
3171    136
3172     85
3173    140
3174    140
3175     54
3176    116
3177     91
3178    135
3179     54
3180    129
3181    137
3182    140
3183    140
3184     83
3185    140
3186    140
3187    133
3188    144
3189    137
3190    134
3191    140
3192    132
3193     71
3194    137
3195    140
3196    117
3197    116
3198    139
Name: pre_clean_len, Length: 3199, dtype: int64

In [7]:
max(df['pre_clean_len'])

148

In [20]:
min(df['pre_clean_len'])

8

In [12]:
df.text[3191]

"@ricardorossello ......#FakeNews critics are working overtime, but we're getting great marks from the people that truly matter! \n#PRStrong🇵🇷"

In [14]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(df.text[3191], 'lxml')
print(example1.get_text())

@ricardorossello ......#FakeNews critics are working overtime, but we're getting great marks from the people that truly matter! 
#PRStrong🇵🇷


In [18]:
import re
re.sub("[^a-zA-Z]", " ", df.text[3191])

' ricardorossello        FakeNews critics are working overtime  but we re getting great marks from the people that truly matter    PRStrong  '

In [21]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

In [25]:
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    #stripped = re.sub(combined_pat, '', souped)
    #try:
    #    clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    #except:
    #    clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", souped)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()



In [28]:
testing = df.text[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['jobless claims fell to their lowest level in years',
 'consumer confidence hits an year high close to breaking the all time record a big jump from last years peop https t co ftdzq lwuu',
 'the democrats are playing a high level con game in their vicious effort to destroy a fine person it is called the https t co ma jxndi',
 'these law enforcement people took the law into their own hands when it came to president trump lindseygrahamsc',
 'rd session of the united nations general assembly unga https t co xjyx rrh f',
 'consumer confidence rose in september notching its highest level in about years the consumer board s index ro https t co ywjgz yu z',
 'remarks by president trump at a luncheon hosted by the secretary general of the united nations https t co wt mdnmdi',
 'remarks by president trump to the rd session of the united nations general assembly https t co oshhko zy https t co fm lsse',
 'rush limbaugh to republicans you can kiss the midterms goodbye if you don t get highly qual